# STAT8107 Data Mining Techniques
## Group Project

**Members (UID):**  
TBC

## Import Modules

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Load Data

In [6]:
# Read csv files
houseprice_train_df = pd.read_csv('trainPrice.csv')
houseprice_test_df = pd.read_csv('testPrice.csv')
subway_df = pd.read_csv('Subways.csv')
schools_df = pd.read_csv('Schools.csv')

## Exploratory Data Analysis (EDA)

In [12]:
houseprice_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 25 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   key                                 1601458 non-null  int64  
 1   apartment_id                        1601458 non-null  int64  
 2   city                                1601458 non-null  int64  
 3   transaction_year_month              1601458 non-null  int64  
 4   transaction_date                    1601458 non-null  object 
 5   year_of_completion                  1601458 non-null  int64  
 6   exclusive_use_area                  1601458 non-null  float64
 7   floor                               1601458 non-null  int64  
 8   latitude                            1601458 non-null  float64
 9   longitude                           1601458 non-null  float64
 10  address_by_law                      1601458 non-null  int64  
 11  total_parki

The housing price taining set has 1.6 M records and 25 columns.

In these 25 columns, 9 columns have float datatype, 12 columns have integer datatype and 4 columns have object (string) datatype.

xx columns have NULL values.

In [15]:
houseprice_train_df[["key"]].shape

(1601458, 1)

In [17]:
houseprice_train_df[["apartment_id"]].drop_duplicates().shape

(3906, 1)

In [21]:
houseprice_train_df[["apartment_id"]].value_counts()

apartment_id
16156           17000
8001            15915
7922             6360
7771             6328
8002             6273
                ...  
316                 1
22241               1
36027               1
36649               1
37469               1
Length: 3906, dtype: int64

In [24]:
houseprice_train_df[houseprice_train_df["apartment_id"] == 16156]

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
310260,310260,16156,1,200807,21~31,2008,98.55,19,37.512753,127.088419,...,21.0,district,cogeneration,13791,126.59,130,4.0,2.0,stairway,1200000000
312109,312109,16156,1,200808,11~20,2008,84.99,26,37.512753,127.088419,...,21.0,district,cogeneration,13781,109.20,1201,3.0,2.0,stairway,906000000
312110,312110,16156,1,200808,11~20,2008,84.99,26,37.512753,127.088419,...,21.0,district,cogeneration,13782,109.35,828,3.0,2.0,stairway,906000000
312111,312111,16156,1,200808,11~20,2008,84.99,26,37.512753,127.088419,...,21.0,district,cogeneration,13783,109.34,436,3.0,2.0,stairway,906000000
312112,312112,16156,1,200808,11~20,2008,84.99,26,37.512753,127.088419,...,21.0,district,cogeneration,13784,109.51,29,3.0,2.0,stairway,906000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600565,1603406,16156,1,201809,1~10,2008,59.99,5,37.512753,127.088419,...,21.0,district,cogeneration,13797,82.10,27,3.0,2.0,stairway,1450000000
1600566,1603407,16156,1,201809,1~10,2008,59.99,5,37.512753,127.088419,...,21.0,district,cogeneration,13798,82.32,104,3.0,2.0,stairway,1450000000
1600567,1603408,16156,1,201809,1~10,2008,59.99,5,37.512753,127.088419,...,21.0,district,cogeneration,13799,82.10,27,3.0,2.0,stairway,1450000000
1600568,1603409,16156,1,201809,1~10,2008,59.99,5,37.512753,127.088419,...,21.0,district,cogeneration,13800,82.13,29,3.0,2.0,stairway,1450000000


In [8]:
houseprice_test_df.shape

(3918, 25)

In [9]:
subway_df.shape

(405, 5)

In [10]:
schools_df.shape

(1921, 9)